In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
from transformers.file_utils import ModelOutput
from dataclasses import dataclass
from typing import Optional, Tuple


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'models/29epoch combined with egg 8051 real865'
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)
sampling_rate = processor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_path).to(device)

In [3]:
x = torch.randn(1, 16000*3, requires_grad=True).cuda()
torch.onnx.export(model, x, 'model.onnx',
                  export_params=True,
                  verbose=True,
                  opset_version=15,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {1 : 'audio_len'}, 'output' : {1 : 'audio_len'}},
                  keep_initializers_as_inputs=False, )

C:\Program Files\Python39\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:578: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
C:\Program Files\Python39\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:617: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz * self.num_heads, tgt_len, self.head_dim):
C:\Program Files\Python39\lib\site-packages\torch\onnx\symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to

graph(%input : Float(1, *, strides=[48000, 1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.0.conv.weight : Float(512, 1, 10, strides=[10, 10, 1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.0.conv.bias : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.1.conv.weight : Float(512, 512, 3, strides=[1536, 3, 1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.1.conv.bias : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_layers.1.layer_norm.weight : Float(512, strides=[1], requires_grad=1, device=cuda:0),
      %wav2vec2.feature_extractor.conv_laye

In [1]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic('model.onnx', 'model_quantized.onnx')

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: <built-in function __import__> returned a result with an error set

Ignore MatMul due to non constant B: /[MatMul_244]
Ignore MatMul due to non constant B: /[MatMul_246]
Ignore MatMul due to non constant B: /[MatMul_344]
Ignore MatMul due to non constant B: /[MatMul_346]
Ignore MatMul due to non constant B: /[MatMul_444]
Ignore MatMul due to non constant B: /[MatMul_446]
Ignore MatMul due to non constant B: /[MatMul_544]
Ignore MatMul due to non constant B: /[MatMul_546]
Ignore MatMul due to non constant B: /[MatMul_644]
Ignore MatMul due to non constant B: /[MatMul_646]
Ignore MatMul due to non constant B: /[MatMul_744]
Ignore MatMul due to non constant B: /[MatMul_746]
Ignore MatMul due to non constant B: /[MatMul_844]
Ignore MatMul due to non constant B: /[MatMul_846]
Ignore MatMul due to non constant B: /[MatMul_944]
Ignore MatMul due to non constant B: /[MatMul_946]
Ignore MatMul due to non constant B: /[MatMul_1044]
Ignore MatMul due to non constant B: /[MatMul_1046]
Ignore MatMul due to non constant B: /[MatMul_1144]
Ignore MatMul due to non con